# ASSIGNMENT 2 : Team 8

## `Assumptions`

* Based in the _USA_ according to all the given county, state and city names. Assuming it's the ONLY country's data

* __VIN__ stands for __Vehicle Identification Number__ and its purpose is to uniquely identify each vehicle in the country (just like a Number Plate). _DOL Vehicle ID_ stands for Department of Licensing Vehicle ID and again, seems that the purpose of it is also to uniquely ID each vehicle in the country

* Although, on checking, VIN is not unique for all the rows while DOL is unique for all rows. Thus, dropping DOLVID as there is no usecase for it since unique for all rows

## `All Imports`

In [12]:
import pandas as pd

## `Loading & Pre-processing Dataset`

In [13]:
MAIN_DATA = pd.read_csv("Electric_Vehicle_Data.csv")

# Total Rows & Rows with NaN (even single one of it)
print("Total rows:", len(MAIN_DATA))
print("Rows with NaN:", len(MAIN_DATA[MAIN_DATA.isnull().any(axis=1)]))
print(
    "\nThus, simply dropping all the rows with even a single NaN value as its almost a negligible number compared to total rows!"
)

# Dropping rows with NaN (even single one of it amongst the columns)
MAIN_DATA = MAIN_DATA.dropna()

# Printing all the total rows now
print("\nTotal rows Now:", len(MAIN_DATA))

# Total unique DOL Vehicle ID
print("Total unique DOL Vehicle ID:", len(MAIN_DATA["DOL Vehicle ID"].unique()))

# Dropping DOLVID column
print(
    "Dropping DOL Vehicle ID column as its unique for all rows (no generalisation possible)!"
)

# Dropping DOLVID as its of no use (unique for all so cant generalise)
MAIN_DATA.drop(columns=["DOL Vehicle ID"], inplace=True)

Total rows: 181458
Rows with NaN: 403

Thus, simply dropping all the rows with even a single NaN value as its almost a negligible number compared to total rows!

Total rows Now: 181055
Total unique DOL Vehicle ID: 181055
Dropping DOL Vehicle ID column as its unique for all rows (no generalisation possible)!


## `Unique Values (for all columns)`

In [14]:
# Renaming the VIN column
MAIN_DATA.rename(columns={"VIN (1-10)": "VIN"}, inplace=True)

# Total unique values for each of the column
print("\nTotal unique values for each of the column:")
for col in MAIN_DATA.columns:
    print(col, ":", len(MAIN_DATA[col].unique()))


Total unique values for each of the column:
VIN : 11055
County : 39
City : 468
State : 1
Postal Code : 542
Model Year : 22
Make : 40
Model : 143
Electric Vehicle Type : 2
Clean Alternative Fuel Vehicle (CAFV) Eligibility : 3
Electric Range : 103
Base MSRP : 31
Legislative District : 49
Vehicle Location : 542
Electric Utility : 75
2020 Census Tract : 1767


## `Dropping columns`

In [15]:
print("\nBelow are the columns that are dropped in the pre-processing part:")

# Removing State as there is ONLY one State unique value
print("-> Removing State as there is ONLY one State unique value!")
MAIN_DATA.drop(columns=["State"], inplace=True)

# Dropping Postal code as its redundant
print("-> Dropping Postal code as its redundant!")
MAIN_DATA.drop(columns=["Postal Code"], inplace=True)

# Dropping Legislative District
print("-> Dropping Legislative District as its not helping in any kind of data generalisation!")
MAIN_DATA.drop(columns=["Legislative District"], inplace=True)

# Dropping Latitude Longitude data as its of no use (redundant)
print("-> Dropping Latitude Longitude data as its of no use (redundant & not generalisable)!")
MAIN_DATA.drop(columns=["Vehicle Location"], inplace=True)

# Making new file for MAIN_DATA (FINAL PRE-PROCESSED DATA)
MAIN_DATA.to_csv("MAIN_DATA.csv")


Below are the columns that are dropped in the pre-processing part:
-> Removing State as there is ONLY one State unique value!
-> Dropping Postal code as its redundant!
-> Dropping Legislative District as its not helping in any kind of data generalisation!
-> Dropping Latitude Longitude data as its of no use (redundant & not generalisable)!


## `Relation b/w City, County & State`

In [16]:
# CODE HERE